# Mathematik für Biologiestudierende II

Sommersemester 2024

07.04.2024

&copy; 2024 Prof. Dr. Rüdiger W. Braun 

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
sns.set_theme()

# Heteroskedastizität

* Die ANOVA vergleicht die Varianzen innerhalb der einzelnen Gruppen mit der Varianz im gesamten Datensatz, um die Unterschiede zwischen den Gruppen zu untersuchen
* À priori geht das erstmal nur, wenn die Varianzen innerhalb der Gruppen gleich sind

* Ein Datensatz ist *heteroskedastisch*, wenn die verschiedenen Gruppen unterschiedlich Varianz haben

# Der Levene-Test

Der Levene-Test testet auf Gleichheit der Varianzen

Beispiel:  Meerschweinchenzähne

* Drei Gruppen von Meerschweinchen, je nach täglicher Gabe an Vitamin C
  * kleine Dosis
  * mittlerer Dosis
  * große Dosis
* Nach 42 Tagen wird die Zahnlänge bestimmt

Quelle: The Statistics of Bioassay

In [ ]:
small_dose = np.array([
    4.2, 11.5, 7.3, 5.8, 6.4, 10, 11.2, 11.2, 5.2, 7,
    15.2, 21.5, 17.6, 9.7, 14.5, 10, 8.2, 9.4, 16.5, 9.7
])

medium_dose = np.array([
    16.5, 16.5, 15.2, 17.3, 22.5, 17.3, 13.6, 14.5, 18.8, 15.5,
    19.7, 23.3, 23.6, 26.4, 20, 25.2, 25.8, 21.2, 14.5, 27.3
])

large_dose = np.array([
    23.6, 18.5, 33.9, 25.5, 26.4, 32.5, 26.7, 21.5, 23.3, 29.5,
    25.5, 26.4, 22.4, 24.5, 24.8, 30.9, 26.4, 27.3, 29.4, 23
])

Test auf Heteroskedastizität

In [ ]:
stats.levene(small_dose, medium_dose, large_dose)

* Der p-Wert ist 0.53.  Hetereskedastizizät kann nicht nachgewiesen werden.

* Wir fahren mit der ANOVA fort

In [ ]:
stats.f_oneway(small_dose, medium_dose, large_dose)

* Gabe von Vitamin C hat Einfluss auf das Zahnwachstum
* Als nächstes würde man eine post-hoc Analyse machen

### Beispiel: Barsche

In [ ]:
df = pd.read_csv('barsche.csv')
df.head()

In [ ]:
sns.boxplot(data=df, x="Art", y="Länge");

Sieht heteroskedastisch aus

In [ ]:
ds = df[df.Art=='gestreift'].Länge
dl = df[df.Art=='gefleckt'].Länge
db = df[df.Art=='blau'].Länge
dr = df[df.Art=='braun'].Länge

In [ ]:
stats.levene(ds, dl, dr, db)

# Probleme beim Test auf Heteroskedastizität

* Die Nullhypothese beim Levene-Test ist 

> $H_0$:  Die Daten sind homoskedastisch

* Ein Hypothesentest "beweist" nie die Nullhypothese
  * bei starken Indizien dagegen lehnt er sie ab
  * bei starken Indizien dafür behält er sie bei
  * bei unklaren Indizien behält er sie auch bei 

* um zu erkennen, ob der Levene-Test Heteroskedastizität überhaupt erkennen kann, wäre eine Poweranalyse für den Levene-Test nötig, das ist aber unrealistisch

* auch das andere Extrem ist möglich:  Der Stichprobenumfang ist so groß, dass kleine Unterschiede schon signifikant werden

Alternativen zum Levene-Test

* Bartlett-Test `stats.bartlett` (wenn völlig klar ist, dass die Daten normalverteilt sind)
* Brown-Forsyth-Test `stats.levene` mit `center=trimmed` (ähnlich`zum Levene-Test)
* Fligner-Test `stats.fligner` (nicht-parametrisch)

# Alexander-Govern-Test

Wenn die Daten heteroskedastisch, aber normalverteilt sind, dann rechnet man einen Alexander-Govern-Test

In [ ]:
stats.alexandergovern(ds, dl, dr, db)

Im homoskedastischen Fall ist der p-Wert des Alexander-Govern-Tests meist schlechter als der von `f_oneway`

In [ ]:
stats.alexandergovern(small_dose, medium_dose, large_dose)

In [ ]:
stats.f_oneway(small_dose, medium_dose, large_dose)  

# Post-hoc Analyse

* Der t-Test kann nur gerechnet werden, wenn die Varianzen der zu vergleichenden Datensätze übereinstimmen
* Im heteroskedastischen Fall ist das nicht der Fall
* Man rechnet dann einen Welch-Test
* In scipy ist der Welch-Test wie folgt implementiert

In [ ]:
stats.ttest_ind(db, dr, equal_var=False)

* Problem:  Arbeitet nicht mit `MultiComparison` zusammen

In [ ]:
# ganz kleines Programm

def welch(x, y):
    return stats.ttest_ind(x, y, equal_var=False)

In [ ]:
from statsmodels.sandbox.stats.multicomp import MultiComparison

In [ ]:
muc = MultiComparison(df.Länge, df.Art)

In [ ]:
muc.allpairtest(welch, method='holm')[0]

# Normalverteilungsannahmen

* Sowohl `f_oneway` als auch `alexandergovern` liefern nur für normalverteilte Daten richtige Ergebnisse
* Normalverteilungsannahmen prüfen wir mit dem Q-Q-Plot

In [ ]:
import statsmodels.api as sm
pp = sm.ProbPlot(db)
pp.qqplot();

ausreichende Übereinstimmung

In [ ]:
df = pd.read_csv('libellen.csv')
df.head()

In [ ]:
df.Art.value_counts()

In [ ]:
dg = df[df.Art=='graue'].Länge
du = df[df.Art=='grüne'].Länge
da = df[df.Art=='ägyptische'].Länge
dB = df[df.Art=='Bilker'].Länge

In [ ]:
pp = sm.ProbPlot(dB)
pp.qqplot();

nicht normalverteilt

# Kruskal-Wallis-Test

im Fall nicht normalverteilter Daten rechnet man den Kruskal-Wallis-Test

In [ ]:
stats.kruskal(dg, du, da, dB)

## Post-hoc Analyse

Das nicht-parametrische Analogon zum unverbundenen t-Test ist der Mann-Whitney-Test

In [ ]:
muc = MultiComparison(df.Länge, df.Art)

In [ ]:
muc.allpairtest(stats.mannwhitneyu, method='holm')[0]

In [ ]:
sns.boxplot(data=df, x='Art', y='Länge');